# Homework 9: Remove Stopwords

I provide a .zip containing .txt and .docx files​

After extracting the .zip contents, use "glob" module to get the file names in a list

For each file, remove punctuation and stop words​

Produce a single .dat file containing the name of each file in quotes, a colon, then a list of words separated by commas​. The list of words per file should be unique for that file. Do not include URLs or phone numbers. Words should be made lowercase. 

Example output:

"File 1.txt" : word1, word2, word3, word7​
"name of file.docx" : word8, word2, word1, word10​
"another file.doc" : word1, word12, word6​

Submit only your .ipynb file.


When you are done with the assignment or have spent an hour on the homework (whichever comes first), please send me an email indicating which of these you have reached​ ​

To: benpayne@umbc.edu​
Subject: Data 601 Spring 2019 Week 9 remove stop words​
Examples of content: "I have spent 1 hour on the homework" or "I am done with the assignment in 20 minutes."​
If you've not completed the homework, please specify what your status is.​

week_09_txt_and_docx.zip+

In [1]:
#install packages
#conda install -c anaconda nltk
#conda install -c conda-forge/label/cf202003 python-docx
#conda install -c conda-forge validators

In [2]:
import zipfile
import re
import nltk
import pandas
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize 
import glob
import docx

In [3]:
#Used https://stackoverflow.com/questions/3451111/unzipping-files-in-python for this
with zipfile.ZipFile("week_09_txt_and_docx.zip","r") as zip_ref:
    zip_ref.extractall("./Homework/") #extracts the files in the zip folder into a homework folder in the working directory

In [4]:
#set the working directory to the one I just created by unzipping the file above
import os
path = os.getcwd()
path
os.chdir('./Homework')
print(os.getcwd())

C:\Users\jcc32\Data Science MPS\Data 601\Homework\Homework


In [5]:
#used https://docs.python.org/3/library/glob.html to figure out the glob library
#Creating a list of the file names
list_of_files = []
for filepath in glob.iglob(r'*'):
    list_of_files.append(filepath)
list_of_files

['52256-0.txt',
 '53031-0.txt',
 '58108-0.txt',
 'blind_text.txt',
 'dr_yawn.txt',
 'how_rubber_goods_are_made.txt',
 'most_boring_ever.txt',
 'most_boring_part2.txt',
 'pg12814.txt',
 'pg14895.txt',
 'pg43994.txt',
 'random_text.txt',
 'smiley_the_bunny.txt',
 'week_10_document1.docx',
 'week_10_document2.docx']

In [6]:
#download the stopwords and punctuation lists that I plan to remove
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcc32\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jcc32\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Now that I've set up the files and laid the groundwork for my libraries, I will iterate through each file and remove the stopwords and punctuation. 

In [7]:
#This loop iterates through the list of files and determines if the file name is a .txt or 
#.docx, then pens that file and appends it to a dictionary with the file name as the key

file_dict = {}
for i in list_of_files:
    if '.txt' in i:
        document = []
        text = open(str(i), "r", encoding="utf8")
        for j in text:
            document.append(j)
        file_dict.update({i:'\n'.join(document)})
    else:
        document = []
        text = docx.Document(str(i))
        for k in text.paragraphs:
            document.append(k.text)
        file_dict.update({i:'\n'.join(document)})


In [8]:
file_dict.keys()

dict_keys(['52256-0.txt', '53031-0.txt', '58108-0.txt', 'blind_text.txt', 'dr_yawn.txt', 'how_rubber_goods_are_made.txt', 'most_boring_ever.txt', 'most_boring_part2.txt', 'pg12814.txt', 'pg14895.txt', 'pg43994.txt', 'random_text.txt', 'smiley_the_bunny.txt', 'week_10_document1.docx', 'week_10_document2.docx'])

Now that I have a dictionary with my files as keys and my text as the values, I will iterate through and tokenize by word. 

In [9]:
for key in file_dict:
    file_dict[key] = word_tokenize(file_dict[key]) #tokenize the text for each key

Next, I will remove URLs, phone numbers, stopwords and punctuation. I used the nltk_for_text_processing.ipynb as a guide for this

In [10]:
#remove phone numbers and URLS using regular expressions
#used https://www.geeksforgeeks.org/python-check-url-string/
phoneRegex = re.compile(r'(\d\d\d-)?\d\d\d-\d\d\d\d')#used 'Automate the Boring Stuff text'
urlregex = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|www.[a-z]+.[a-z]+')

for key in file_dict: 
    new_list = []
    for word in file_dict[key]:
        matchphone = phoneRegex.search(word) #searches string for a phone number matching the regex pattern
        if matchphone is None:
            new_list.append(word) #only adds those not matching the pattern to the new list
    file_dict[key] = new_list
    
for key in file_dict:#repeat for URL patterns
    new_list = []
    for word in file_dict[key]:
        matchurl = urlregex.search(word)
        if matchurl is None:
            new_list.append(word)
    file_dict[key] = new_list

In [11]:
#create list of stopword
en_stops= set(stopwords.words('english'))
#en_stops

In [12]:
#iterate through the dictionary and filter out stopwords. 
#Convert all words to lowercase for consistency
for key in file_dict:
    filtered_text = []
    for word in file_dict[key]:
        if word not in en_stops:
            filtered_text.append(word.lower())
    file_dict[key] = filtered_text

In [13]:
#Replace all punctuation with a space, filter out empty strings

def remove_punct(mydict):
    for key in mydict:
        new_list = []
        for word in mydict[key]:
            if word.isalpha() == True:
                new_list.append(word)
        mydict[key] = new_list
    return mydict
file_dict2 = remove_punct(file_dict)

In [14]:
#remove empty strings
for key in file_dict2:
    new_list = []
    for word in file_dict2[key]:
        if word != '\s':
            new_list.append(word)
    file_dict2[key] = new_list
    

In [15]:
for key in file_dict2:
    filtered_text = []
    for word in file_dict2[key]:
        if word not in en_stops:
            filtered_text.append(word.lower()) #all words are lower case
    file_dict2[key] = filtered_text

Now that I've removed unwanted punctuation, stopwords, URLs and Phone numbers, I will find the word frequencies for each document 

In [16]:
#Here I create a new dictionary containing the frequencies for each word used in each document. The keys represent the
#document and the values are a nested dictionary containing word frequencies
Freq_dict = {}
for key in file_dict2:
    fdist = nltk.FreqDist()
    for word in file_dict2[key]:
        fdist[word] += 1
    Freq_dict.update({key:fdist})

In [17]:
Freq_dict['52256-0.txt']['project']

88

Last step, I need to write this dictionary to a .dat file in the format Key: word1, word2,...

In [18]:
#I got this fuction from: https://www.quora.com/How-do-I-write-a-dictionary-to-a-file-in-Python

def write_dic(d,path): #you can also provide additional parameter for path
    f=open (path,"w")
    keys=d.keys()
    for k in keys:
        f.write('"' + str(k)+ '"' + ' : ' + str(d[k])[1:-1].replace("'", "")) #you can also provide formatting
        f.write("\n")
    f.close()

In [19]:
write_dic(file_dict2, './myfile.dat')

In [20]:
#and for fun, the word frequency file
def write_dic(d,path): #you can also provide additional parameter for path
    f=open (path,"w")
    keys=d.keys()
    for k in keys:
        sec_keys = d[k].keys()
        for j in sec_keys:
            f.write('"' + str(k)+ '"' + ' : ' + str(j) + ' : ' + str(d[k][j])) #you can also provide formatting
            f.write("\n")
    f.close()
write_dic(Freq_dict, './FreqFile.dat')